In [1]:
# import of important libraries
from datetime import datetime
from datetime import timedelta
import time
import random
import re
from functions import load_csv, write_csv, sort
import csv
import sys

In [2]:
# Before describing all the classes, one can take a look in the functions.py file which also uses some self-built functions that will be used throughout the program.

# PART 1: CLASSES

#### Appointment

In [3]:
# starting with appointment_sched class, it has as attributes a list of appointment's where appointment itself is a class within the appointment_sched class which has as attributes a type which is a date in the conext here, a health professional member and a patient which is a list of information. When a new appoitnment instance is called, it is possible to use also some methods. Add_appoint adds an appointment to the appointment list, del_appoint deletes an appointment from the appointment list and find_appoint finds the next suitable appointment.

class appointment_sched:
  
    list1 = []
    class appointment:

        def __init__(self, type, health_pro,patient):
            self.type = type
            self.health_pro = health_pro
            self.patient = patient

        # add_appoint simply adds a new appointment instance (gathered into a list) to the list of all appointments (No check).
        def add_appoint(self):
            appointment_sched.list1.append([self.type,self.health_pro,self.patient[0],self.patient[1],self.patient[2]])

        # the del_appoint method checks whether the mentioned type, staff and patient is in the appointment_sched list called "list1". If so, then it saves the index in a variable "index" and the counter becomes 1. If the counter is at the end still 0, then a message will be shown to indicate that no appointment has been found. Otherwise, the appointment will be removed from the schedule list. 
        def del_appoint(self):
            counter = 0
            index = 0
            for i in range(len(appointment_sched.list1)):
                if appointment_sched.list1[i][0] == str(self.type) and appointment_sched.list1[i][1] == self.health_pro and appointment_sched.list1[i][2] == self.patient[0]:
                    index = i
                    counter += 1
            if counter == 0:
                print("Appointment not found. Please restart again. ")
            if counter == 1:
                appointment_sched.list1.pop(index)

        # the find_appoint method works with the datetime library and simply tries to find a suitable date that fits in the doctors schedule. It mainly differentiates between 3 cases. Appointment wishes after 18 o' clock should be delayed to the next day (08:00), the same is true for the lunch break (12:30 should be delayed to 13:30) and all other blocked dates (where is already a meeting for the doctor) should lead to proposal for the next meeting (00:30 later). This whole processes will be done until a free date is found (see the while loop which only terminates when the counter is 1 (i.e. if a date is found)). 
        def find_appoint(self):

            counter = 0
            while counter == 0:
                for i in range(0,len(appointment_sched.list1)):
                    if self.type.hour == 18:
                        self.type = self.type + timedelta(days = 1, hours = -10)
                        continue
                    if self.type.hour == 12 and self.type.minute == 30:
                        self.type = self.type + timedelta(hours = 1)
                        continue
                    if appointment_sched.list1[i][0] == str(self.type) and appointment_sched.list1[i][1] == self.health_pro:
                        n = 30
                        self.type = self.type + timedelta(minutes=n)
                        continue
                counter = 1
            return self.type

#### Patient

In [4]:
# the patient class has three attributes (name,address,phone) and these attributes become powerful in using one of the two methods. request_repeat is a method that helps to show a doctor the previous prescription of a patient so that he can decide whether to continue or not with one of them. request_appoint is directly connected to the make_appoint method in the receptionist class which at the end enables to call the _add_appoint or del_appoint method of the appointment_sched class so that meetings can be added or removed from the schedule list. 

class patient:

    def __init__(self, name, address, phone):

        self.name = name
        self.address = address
        self.phone = phone

    # this method takes an existing prescription list and an employee number as arguments and checks for the mentioned patient and the corresponding doctor whether there are some previous prescription in the list. If so, the issue_presc of the doc class is called.
    def request_repeat(self,list_p,e_number):    
        counter = 0
        tmp = []
        for i in range(0,len(list_p)):
            if list_p[i][1] == self.name and list_p[i][2] == self.address and list_p[i][3] == self.phone and list_p[i][4] == e_number:
                tmp.append(list_p[i])
                counter += 1
        if counter == 0:
            return None
        else: 
            return tmp

    # this method differentiates between two cases which depend on the value of the given variable "tmp". If this variable is "1" then it calls the make_appoint method of the receptionist class. If this variable is equal to "2" then it calls cancel_appoint of the receptionist class.
    def request_appoint(self,tmp,list_health,list_appoint):
        
        for i in list_health:
            print(i)
        staff = input('Above you can see all the doctors that work in our practice. Please search for your doctor and enter the number that appears to the most left. : ')
        if tmp == '1':
            receptionist.make_appoint(list_appoint,staff,[self.name,self.address,self.phone])
        if tmp == '2':
            receptionist.cancel_appoint(list_appoint,staff,[self.name,self.address,self.phone])

#### Health Professional

In [5]:
# this class is used in the health_pro class later and it essentially returns the key and value section of a dictionary.
class IterMixin(object):
    def __iter__(self):
        for attr, value in self.__dict__.items():
            yield attr, value

# health pro class consists of 2 attributes (name, employee number). The consultation method will not be used in the program here but it could be interesting for future purpose. It essentially takes a summary text of a consultation and then searches for one key word, which should be one included in the list of possible consultations seen above. If it finds one, then it returns it. 
class health_pro(IterMixin):

    def __init__(self, name, employ_number):
        self._name = name
        self._employ_number = employ_number

    def consultation(text):
        list = re.sub("[^\w\s]", "", text).split()
        print(text)
        for i in possible_consultations:
            for j in list:
                if i == j:
                    return i
                
# the properties below restrict a health_pro in its abilities to work with the attributes. In other words, a health_pro instance should only be able to be read and not be changed (no setter option).                   
    @property
    def name(self):
        return self._name
        
    @property
    def number(self):
        return self._employ_number


#### Doctor and Nurse

In [6]:
# the doc class stands for a doctor and it inherites all the attributes and methods of the health_pro class.
class doc(health_pro):

    def __init__(self, name, employ_number):
        super().__init__(name,employ_number)
    
    # Beside all the methods of the health_pro class, a doctor class also has this authorization method which simply allows to check if the mentioned employee number is a doctor and therefore he is authorized to issue repeated prescriptions.
    def authorization(self,list):
        for i in list:
            if i[0] == self.number and i[2] == "doc":
                return True
        return False

    # this methods takes a list as argument and this list will be shown to the doctor (including an index number) so that he can decide which one he wants to choose and hence to issue to the patient. 
    def issue_presc(list):
        for i in range(0,len(list)):
            list[i] = [i] + list[i]
        for i in range(0, len(list)):
            if str(list[i][0]) == issue:
                return list[i]          

# the nurse class inherites all the attributes and methods of the health_pro class but it has no own attribute + methods.
class nurse(health_pro):
    def __init__(self, name, employ_number, docORnurse,count):
        super().__init__(name,employ_number,docORnurse,count)

#### Prescription

In [7]:
# the prescription class has five attributes: a name for the prescription ("type"), a patient tuple, a doctor (only employee number), a quantity and a dosage of the prescription. There are no methods within the class and it will therefore never be used throughout the main program. Nevertheless, futur purpose easily allow to reuse and adapte the existing class of "prescription". 
class prescription:

    def __init__(self, type, patient, doc, quantity, dosage):
        self.type = type
        self.patient = patient
        self.doc = doc
        self.quantity = quantity
        self.dosage = dosage    

#### Receptionist

In [8]:
# the receptionist class contains as attributes the employee name and his number and it consists of two main methods: make _appoint and cancel_appoint. Beside these two methods, there are also properties (only read option) and two further methods whereas authorization is similar to the authorization method in the doctor class (only that now a receptionist sould be checked) and the find_a_doc method allows to check for available doctors that can still take new patients. 
class receptionist:

    def __init__(self, name, employ_number):
        self._name = name
        self._employ_number = employ_number

    # As already seen in the health_pro class, here also a receptionist instance should only be readable and not possible to change. (Static csv files)
    @property
    def name(self):
        return self._name
        
    @property
    def number(self):
        return self._employ_number

    # Similar to auhtorization in doc class
    def authorization(self,list):
        for i in list:
            if i[0] == self.number and i[2] == "receptionist":
                return True
        return False
    
    # The idea here is to check which doctor still has availabilities with respect to new patients. Therefore it needs a patient and a healt_pro list and it loops over all existing patients and looks up which health_pro they have and counts them. After this process, it checks the health_pro list for doctors/nurse which still have less than 500 patients and returns the first one that it finds.
    def find_a_doc(pat_file,length_pat,health_file,length_health,reference_number):
        tmp = []
        for i in load_csv(health_file,length_health):
            i.append(0)
            tmp.append(i)
        for i in load_csv(pat_file,length_pat):
            for j in range(0,len(tmp)):
                if j+1 == int(i[-1]):
                    tmp[j][-1] += 1
        for i in range(0,len(tmp)):
            if int(tmp[i][-1]) < int(reference_number) and tmp[i][2] == "doc":
                return tmp[i][0]

    # this method always starts in finding a suitable appointment. For this it asks the patient which is his preferred date and time. After this, the date1 variable does some concatenation and then transforms it into a datetime format. At the end it starts the request whether the mentioned date is available. If not, the find_appoint function of the appointment_sched class will return the next available date for the doctor. The final step in make_appoint is then to ask the patient if he agrees with the proposed date or not. If not, the while loop continues. Otherwise, the while loop will be left and the receptionist did his work and the appointment can finally be added to the schedule.     
    def make_appoint(list,health_pro,patient):
        question = "2"
        while question == "2":
            tmp1d = input('Please enter the date (m/d/y) of your preferred appointment : ')
            tmp1t = input('Please enter the time (h:m) of your preferred appointment : ') 
            date1 = datetime.strptime(tmp1d + ' ' + tmp1t + ':' + '00.000', '%m/%d/%Y %H:%M:%S.%f')
            date1 = appointment_sched.appointment(date1,health_pro,patient).find_appoint()
            print(date1)
            question = input("The next available appointment is the one seen above. Do you agree (enter 1) or do you want that we continue to search (enter 2)? ") 
        appointment_sched.appointment(date1,health_pro,patient).add_appoint()

    # cancel_appoint works similar but it does search for the appointment that the patient mentiones. It simply sends (after the date transformation) a del_appoint request and this request checks whether an entry was found or not. If not, a failure message will be shown to the receptionist.    
    def cancel_appoint(list,health_pro,patient):
        tmp1d = input('Please enter the date (m/d/y) of your appointment : ')
        tmp1t = input('Please enter the time (h:m) of your appointment : ')  
        date1 = datetime.strptime(tmp1d + ' ' + tmp1t + ':' + '00.000', '%m/%d/%Y %H:%M:%S.%f')
        appointment_sched.appointment(date1,health_pro,patient).del_appoint()

# PART 2: MAIN

In [9]:
# It begins in reading all the data files that are important to run the app. These are a health professional, an appointment schedule, a patient and a prescription file whereas only the appointment schedule and the patient file are dynamic files in this program. The other two files represent static files and are only used for information delivery. To read the files, the csv library is used and it opens the file with "with", reads every line with "csv.reader" and uses then a for loop to get every value in one line which is seperated in the csv file by a "," and appends this to a temporary list. If every value in one line is read, then the temporary list (list_tmp) will be added to the main list (list_health or list_patient or list_presc).
list_health = load_csv('health_pro.csv',3)

# Add a counter variable to health_pro list that counts the number of patients per health_pro. 
for i in list_health:
    i.append(0)

# now, it asks the user to mention a patient file. It then loads the corresponding file. After this, it already counts the number of patients per doctor. For this, it loops over all the patients in the patient list and all the doctors in the doctor list and checks whether the employee number is equal to the corresponding doctor. If so, the count of that doctor will be increased bu one unit.
file_pat = input('Please enter the name of your patient file (without .csv): ')
list_patient = load_csv(file_pat + '.csv',4)
for i in list_patient:
    for j in range(0,len(list_health)):
        if j+1 == int(i[3]):
            list_health[j][3] = int(list_health[j][3]) + 1

# Similar to the patient file, the user will be asked to mention the prescription file for which he wants to test for. After this, load_csv is again used for reading the csv file.
list_presc = []
file_presc = input('Please enter the name of your prescription file (without .csv): ')
list_presc = load_csv(file_presc + '.csv',7)

# Finally, the user has to metnion the appointment file.
file_appoint = input('Please enter the name of your appointment file (without .csv): ')
appointment_sched.list1 = load_csv(file_appoint + '.csv',5)
sorted = sort(appointment_sched.list1,0,1)

# check 1 - next available date (+ 00:30 when date is already blocked)

# last_date takes the last date of the sorted appointment list and saves only the the date of it
last_date = datetime.strptime(sorted[-1][0], '%Y-%m-%d %H:%M:%S')

# staff1 also takes the last date and of the sorted appointment list and saves the employee
staff1 = sorted[-1][1]

# Now, calling the find_appoint method of the appointment_sched and asking for an appointment on the same date as last_date should lead to an appointment that starts later than last_date. This will be tested with unittest in "CheckNextDate" below.
next = appointment_sched.appointment(last_date,staff1,('test','test','test')).find_appoint()
for i in sorted:
    i[0] = datetime.strptime(i[0], '%Y-%m-%d %H:%M:%S')
last_date = time.mktime(last_date.timetuple())
next = time.mktime(next.timetuple())

# Use unittest and check that "last_date" is greater than "next"
import unittest
class CheckNextDate(unittest.TestCase): 
    def test_next_date(self): 
        self.assertGreater(next, last_date)

# check 2 - next available date (lunch break at 13:00 until 14:00)

# the idea is similar to the one of check 1. It first adds a new appointment in "nexta" which starts at 12:30 (the time before lunch) and then once again tries to add a second appointment at the same time and date (in calling once again find_appoint). the result should lead to a date that starts at 13:30. This will be tested below with unittest in CheckEquality1.
nexta = appointment_sched.appointment(datetime.strptime('2022-01-01 12:30:00', '%Y-%m-%d %H:%M:%S'),'new_staff',('test2a','test2a','test2a')).find_appoint()
nextb = appointment_sched.appointment(datetime.strptime('2022-01-01 12:30:00', '%Y-%m-%d %H:%M:%S'),'new_staff',('test2b','test2b','test2b')).find_appoint()

# Use unittest and check that "nextb" is equal to '2022-01-01 13:30:00' (a meeting after lunch)
class CheckEquality1(unittest.TestCase):
    def test_date1_date2(self):
        self.assertEqual(str(nextb), '2022-01-01 13:30:00')

# check 3 - next available date (last meeting for the day at 18:00, then the next one is tomorrow at 08:00)           

# the idea is amlost equal to the one in check 2. This time it adds a new appointment in "nextc" which starts with the last meeting of a date (18:00) and then tries the a second one at the same time and date. This should lead to an appointment which starts at 08:00 the next day. This will be tested with unittest in "CheckEquality2" below.
nextc = appointment_sched.appointment(datetime.strptime('2022-01-01 18:00:00', '%Y-%m-%d %H:%M:%S'),'new_staff',('test2a','test2a','test2a')).find_appoint()
nextd = appointment_sched.appointment(datetime.strptime('2022-01-01 18:00:00', '%Y-%m-%d %H:%M:%S'),'new_staff',('test2b','test2b','test2b')).find_appoint()

# Use unittest and check that "nextb" is equal to '2022-01-02 08:00:00' (a meeting the next day)
class CheckEquality2(unittest.TestCase):
    def test_date1_date2(self):
        self.assertEqual(str(nextd), '2022-01-02 08:00:00')
        
# check 4 - authorizatiion

# It first asks the employee for its name and number. After this the auhtorization method of the receptionist and the doch class will be called and if the return "True" a counter variable will be increased by 1 unit. At the end a unittest will be run that checks that the counter will always be less than 2 (i.e. an employee can only be authorized to one class).
e_name1 = input('You must be an employee of our surgery. Please enter your given name: ').lower()
e_name2 = input('You must be an employee of our surgery. Please enter your last name: ').lower()
e_name = e_name1 + e_name2
e_number = input('You must be an employee of our surgery. Please enter your employee number: ').lower()  
counter = 0
if receptionist(e_name,e_number).authorization(list_health) == True:
    counter += 1
if doc(e_name,e_number).authorization(list_health) == True:
    counter += 1
           
# Use unittest and check that every employee is at most in one of the two groups ('receptionist' or 'doc'). In other words, the counter is always smaller than 2.
class CheckSmaller(unittest.TestCase):
    def setUp(self):
        self.counter = 0
    def tearDown(self):
        self.counter = 0
    def test_int1_int2(self):
        self.assertLess(self.counter, 2)

# Check 5 - prescription

# the test is very simple. Calling the request_repeat method of the patient class for every patient (see the loop) and adding up the length of all the outputs should lead to the same result as counting the length of all the prescriptions. This will be tested with unittest in "CheckEqualityInt".
counter = 0
for i in range(0,len(list_patient)):
    enumber = list_patient[i][3]
    if patient(list_patient[i][0],list_patient[i][1],list_patient[i][2]).request_repeat(list_presc,enumber) != None:
        counter += len(patient(list_patient[i][0],list_patient[i][1],list_patient[i][2]).request_repeat(list_presc,enumber))
list_tmp2 = []
for i in range(0,len(list_presc)):
    list_tmp2.append([list_presc[i][1],list_presc[i][2],list_presc[i][3],list_presc[i][4]])

# Use unittest and check for equality (calling request repeat for every patient should be equal to the number of issued prescription)
class CheckEqualityInt(unittest.TestCase):
    def test_int1_int2(self):
        self.assertEqual(counter, len(list_tmp2))
unittest.main(argv=['first-arg-is-ignored'], exit=False)        


Please enter the name of your patient file (without .csv): pat_testdata3
Please enter the name of your prescription file (without .csv): presc_testdata3
Please enter the name of your appointment file (without .csv): appoint_testdata3
You must be an employee of our surgery. Please enter your given name: johnny
You must be an employee of our surgery. Please enter your last name: money
You must be an employee of our surgery. Please enter your employee number: 10


.....
----------------------------------------------------------------------
Ran 5 tests in 0.003s

OK
